# Toronto Neighbourhood Capstone Project

## Part I - Download, clean data, and Present as per the assignment

- Import libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

- Download data and read into dataframe

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
soup = BeautifulSoup(response.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [3]:
table_rows = table.find_all('tr')
rows_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    rows_list.append(row)
    print(row)

[]
['M1A', 'Not assigned', 'Not assigned\n']
['M2A', 'Not assigned', 'Not assigned\n']
['M3A', 'North York', 'Parkwoods\n']
['M4A', 'North York', 'Victoria Village\n']
['M5A', 'Downtown Toronto', 'Harbourfront\n']
['M6A', 'North York', 'Lawrence Heights\n']
['M6A', 'North York', 'Lawrence Manor\n']
['M7A', 'Downtown Toronto', "Queen's Park\n"]
['M8A', 'Not assigned', 'Not assigned\n']
['M9A', 'Etobicoke', 'Islington Avenue\n']
['M1B', 'Scarborough', 'Rouge\n']
['M1B', 'Scarborough', 'Malvern\n']
['M2B', 'Not assigned', 'Not assigned\n']
['M3B', 'North York', 'Don Mills North\n']
['M4B', 'East York', 'Woodbine Gardens\n']
['M4B', 'East York', 'Parkview Hill\n']
['M5B', 'Downtown Toronto', 'Ryerson\n']
['M5B', 'Downtown Toronto', 'Garden District\n']
['M6B', 'North York', 'Glencairn\n']
['M7B', 'Not assigned', 'Not assigned\n']
['M8B', 'Not assigned', 'Not assigned\n']
['M9B', 'Etobicoke', 'Cloverdale\n']
['M9B', 'Etobicoke', 'Islington\n']
['M9B', 'Etobicoke', 'Martin Grove\n']
['M9B', 

- Cleaning of data

In [4]:
rows_list[0:5]

[[],
 ['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n']]

In [5]:
df_neigh = pd.DataFrame(rows_list)
df_neigh.head(5)

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


- Eliminating rows having 'Not Assigned'as data

In [6]:
df_neigh.columns = ['PostalCode', 'Borough','Neighborhood']
df_neigh.drop(0, inplace = True)
df_neigh.drop(df_neigh.loc[df_neigh['Borough']=='Not assigned'].index, inplace=True)
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n


In [7]:
df_neigh['Neighborhood'] = df_neigh['Neighborhood'].map(lambda x: x.rstrip('\n'))
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


- Grouping into Boroughs

In [8]:
df_tor = df_neigh.astype(str).groupby('PostalCode').agg(lambda x: ','.join(x.unique()))
df_tor.reset_index(inplace = True) 
df_tor.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_tor['Neighborhood'].replace("Not assigned", np.nan, inplace = True)
df_tor.Neighborhood.fillna(df_tor.Borough, inplace=True)
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


- Sorting by neighbourhoods

In [10]:
toronto_set = df_tor.sort_values(by=['Neighborhood'])
toronto_set.head(10)

,PostalCode,Borough,Neighborhood
58,M5H,Downtown Toronto,"Adelaide,King,Richmond"
12,M1S,Scarborough,Agincourt
14,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
89,M8W,Etobicoke,"Alderwood,Long Branch"
28,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights"
19,M2K,North York,Bayview Village
62,M5M,North York,"Bedford Park,Lawrence Manor East"
56,M5E,Downtown Toronto,Berczy Park
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


- Calculating the size of the dataset

In [11]:
toronto_set.shape

(103, 3)

## Part II of the Assignment: Adding the Geospatial data

- Import libraries

In [13]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

- instal and import folium

In [14]:
!conda install folium -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.4.0-py_0        conda-forge
    folium:          

In [15]:
import folium

- Download data 

In [16]:
!wget -O Geospatial_data.csv https://cocl.us/Geospatial_data

--2020-03-02 05:29:05--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-02 05:29:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-02 05:29:06--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2sm

- Read data into dataframe

In [18]:
df_cord = pd.read_csv('Geospatial_data.csv')
df_cord.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_tor.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
df_cord.shape
df_tor.shape

(103, 3)

Both the tables' data were found to be of same size, hence they can be merged

In [21]:
df_toronto = pd.concat([df_tor, df_cord], axis = 1)
df_toronto.head(5)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## Part III: Download map and explore clusters

- Use geopy to get latitude and longitude

In [22]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


- find the map

In [23]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

- Explore data in Scarbourough Borough

In [25]:
scarborough_data = df_toronto[df_toronto['Borough'
] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


- Get the geographical loation of Scarborough

In [27]:
address = 'Scarborough, Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Scarborough,Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Scarborough,Toronto are 43.773077, -79.257774.


- CREATING MAP of SCARBOROUGH

In [28]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)
map_scarborough

- Use Foursquare Credentials to segment the above 

In [30]:
CLIENT_ID = 'XZ20NXF0P14MW0QPBEYAXOR2K1YDAVCFP22YN3IGNTKHGGZ5' # your Foursquare ID
CLIENT_SECRET = 'IP5GBIKY3153FGDEKIZBJYXMHXLFEQIQP2VFQJAUTJO2XHE2' # your Foursquare Secret
VERSION = '20190623' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XZ20NXF0P14MW0QPBEYAXOR2K1YDAVCFP22YN3IGNTKHGGZ5
CLIENT_SECRET:IP5GBIKY3153FGDEKIZBJYXMHXLFEQIQP2VFQJAUTJO2XHE2


In [31]:
scarborough_data.loc[4, 'Neighborhood']

'Cedarbrae'

In [33]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


- Get the 50 venues in a radius of 500 meters

In [34]:
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XZ20NXF0P14MW0QPBEYAXOR2K1YDAVCFP22YN3IGNTKHGGZ5&client_secret=IP5GBIKY3153FGDEKIZBJYXMHXLFEQIQP2VFQJAUTJO2XHE2&v=20190623&ll=43.806686299999996,-79.19435340000001&radius=500&limit=50'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5ca92198205d001b0dbeee'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [38]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.
